## Introduction to Data Science - NBA Fantasy Sports

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/current_allplayer_stats_02112017.csv')

In [3]:
df.head()

,Player,Pos,Tm,Rk.x,Age.x,G.x,GS,MP.x,FG,FGA,...,X.,OWS,DWS,WS,WS.48,X..1,OBPM,DBPM,BPM,VORP
0,A.J. Hammons,C,DAL,163,24,17,0,4.5,0.5,1.3,...,NaN,-0.1,0.1,0.0,0.014,NaN,-6.7,1.1,-5.6,-0.1
1,Aaron Brooks,PG,IND,57,32,45,0,14.5,2.0,5.0,...,NaN,-0.2,0.5,0.3,0.019,NaN,-1.8,-2.1,-4.0,-0.3
2,Aaron Gordon,SF,ORL,150,21,54,46,27.5,4.3,10.1,...,NaN,0.6,1.1,1.7,0.055,NaN,-0.9,-0.5,-1.4,0.2
3,Aaron Harrison,SG,CHO,172,22,5,0,3.4,0.0,0.8,...,NaN,-0.1,0.0,0.0,-0.136,NaN,-9.3,-1.7,-11.0,0.0
4,Adreian Payne,PF,MIN,337,25,12,0,7.7,1.4,3.3,...,NaN,0.0,0.1,0.1,0.077,NaN,-2.4,-0.5,-2.9,0.0
